# Test des fonctions de l'API OpenAI avec OpenFisca

In [13]:
from decouple import Config, RepositoryEnv
import openai
import json
from IPython.display import Markdown
import requests

Lecture de la clef d'API

In [33]:
DOTENV_FILE = '.env'
env_config = Config(RepositoryEnv(DOTENV_FILE))
openai.api_key = env_config('OPENAI_API_KEY')

## Définition de la fonction

In [23]:
generate_tax_reform = [
    {
        'name': 'generate_tax_reform',
        'description': """Generate a tax reform proposal using OpenFisca Parameters.

        """,
        'parameters': {
            'type': 'object',
            'properties': {
                'Reform description': {
                    'type': 'string',
                    'description': 'Description of the reform to generate.'
                },
                'parameters_change': {
                    
                    "type": "array",
                    "items": {
                        'type': 'object',
                        'properties':
                        {
                            'parameter_description': {
                                'type': 'string',
                                'description': 'Description of the parameter to change.'
                            },
                            'parameter_name': {
                                'type': 'string',
                                'description': 'OpenFisca parameter name.'
                            },
                            'value': {
                                'type': 'string',
                                'description': 'New value of the parameter. Must be a unique value, not a mathematical operation.'
                            },
                        },
                    }
                }
            }
        }
    }
]

## Préparation du prompt

In [24]:
reform1 = "Je souhaite réduire de 10% la CSG imposable pour les personnes qui travail et de 5% la CSG déductible."

prompt1 = f"""
You are an lawer at french national assembly. You are asked to reform the CSG taxes by a deputy.
You can change the following parameters in the law:
For the "Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité" the OpenFisca parameter name is "prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux" and the default value is 0.068
For the "Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité" the OpenFisca parameter name is "prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux" and the default value is 0.024
Here is what the deputy said:
{reform1}
"""


## Appel de l'API OpenAI

In [28]:
reforms = [prompt1]
for sample in reforms:
    response = openai.ChatCompletion.create(
        model = 'gpt-4',  # gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = generate_tax_reform,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response['choices'][0]['message']['function_call']['arguments'])
    print(json_response)

{'Reform description': "Redesign of CSG taxes as per Deputy's request. CSG imposable reduced by 10% and CSG deductible reduced by 5% for working individuals.", 'parameters_change': [{'parameter_description': "Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité", 'parameter_name': 'prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux', 'value': '0.0646'}, {'parameter_description': "Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité", 'parameter_name': 'prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux', 'value': '0.0216'}]}


In [29]:
json_response

{'Reform description': "Redesign of CSG taxes as per Deputy's request. CSG imposable reduced by 10% and CSG deductible reduced by 5% for working individuals.",
 'parameters_change': [{'parameter_description': "Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité",
   'parameter_name': 'prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux',
   'value': '0.0646'},
  {'parameter_description': "Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité",
   'parameter_name': 'prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux',
   'value': '0.0216'}]}

## Appel de l'API LexImpact

In [44]:
api_payload = {
    "amendement": {},
    "base": 2024,
    "plf": 2024,
    "output_variables": [
        "csg_deductible_salaire",
        "csg_imposable_salaire"
    ],
}
for reform in json_response["parameters_change"]:
    api_payload['amendement'][reform["parameter_name"]]= {
        "start": "2024-01-01",
        "type": "parameter",
        "variable": "csg_salaire",
        "value": float(reform["value"])
    }
api_payload

{'amendement': {'prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux': {'start': '2024-01-01',
   'type': 'parameter',
   'variable': 'csg_salaire',
   'value': 0.0646},
  'prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux': {'start': '2024-01-01',
   'type': 'parameter',
   'variable': 'csg_salaire',
   'value': 0.0216}},
 'base': 2024,
 'plf': 2024,
 'output_variables': ['csg_deductible_salaire', 'csg_imposable_salaire']}

In [45]:
url = env_config('LEX_API_URL') + "/state_simulation"
headers = {'Content-Type': 'application/json','jwt-token': env_config('LEX_TOKEN')}
response = requests.post(url, headers=headers, data=json.dumps(api_payload), cookies=cookie)
response.json()

{'result': {'base': {'state_budget': {'csg_deductible_salaire': -64304294891.978134,
    'csg_imposable_salaire': -22695633461.508907},
   'quantiles': None,
   'compare_before_after': None},
  'plf': {'state_budget': {'csg_deductible_salaire': -64304294891.978134,
    'csg_imposable_salaire': -22695633461.508907},
   'quantiles': None,
   'compare_before_after': None},
  'amendement': {'state_budget': {'csg_deductible_salaire': -61089080024.706024,
    'csg_imposable_salaire': -20426070169.844734},
   'quantiles': None,
   'compare_before_after': None}},
 'errors': []}

## Rédaction de l'amendement

In [38]:
def get_completion(prompt, model="gpt-4", temperature=0):
    """
    model : "gpt-3.5-turbo" | "gpt-4"
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [50]:
response_prompt = f"""
Vous êtes un député à l'assemblée nationale. Vous êtes chargé de réformer la CSG.
Votre assistant informaticien vous a préparé un amendement que vous pouvez déposer à l'assemblée nationale.
Le Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité" est actuellement de 6,8%.
Le Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité correspond à 2,4% du salaire brut.

Voici le contenu de l'amendement en JSON :
{json_response}

Voici son impact sur les finances publiques qui doit être mentionné dans l'exposé des motifs de l'amendement :
- Montant en euro avant réforme : {response.json()['result']['base']['state_budget']}
- Le montant en euro après application du Projet de Loi de Finances proposé par le gouvernement : {response.json()['result']['plf']['state_budget']} 
- Le montant après votre amendement : {response.json()['result']['amendement']['state_budget']}

Vous devez rédiger l'amendement en français.
"""
Markdown(response_prompt)



Vous êtes un député à l'assemblée nationale. Vous êtes chargé de réformer la CSG.
Votre assistant informaticien vous a préparé un amendement que vous pouvez déposer à l'assemblée nationale.
Le Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité" est actuellement de 6,8%.
Le Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité correspond à 2,4% du salaire brut.

Voici le contenu de l'amendement en JSON :
{'Reform description': "Redesign of CSG taxes as per Deputy's request. CSG imposable reduced by 10% and CSG deductible reduced by 5% for working individuals.", 'parameters_change': [{'parameter_description': "Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité", 'parameter_name': 'prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux', 'value': '0.0646'}, {'parameter_description': "Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité", 'parameter_name': 'prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux', 'value': '0.0216'}]}

Voici son impact sur les finances publiques qui doit être mentionné dans l'exposé des motifs de l'amendement :
- Montant en euro avant réforme : {'csg_deductible_salaire': -64304294891.978134, 'csg_imposable_salaire': -22695633461.508907}
- Le montant en euro après application du Projet de Loi de Finances proposé par le gouvernement : {'csg_deductible_salaire': -64304294891.978134, 'csg_imposable_salaire': -22695633461.508907} 
- Le montant après votre amendement : {'csg_deductible_salaire': -61089080024.706024, 'csg_imposable_salaire': -20426070169.844734}

Vous devez rédiger l'amendement en français.


In [51]:
amendement = get_completion(response_prompt, model="gpt-4", temperature=0.5)
Markdown(amendement)

AMENDEMENT

Article additionnel après l'article 10

Objet : Réforme de la Contribution Sociale Généralisée (CSG) sur les revenus d'activité

Exposé des motifs :

La Contribution Sociale Généralisée (CSG) est un impôt qui pèse lourdement sur les revenus d'activité des travailleurs. Afin d'alléger cette charge, il est proposé de réduire le taux de la CSG déductible de 5% et le taux de la CSG imposable de 10%.

Avant la réforme, le montant de la CSG déductible sur les salaires s'élevait à -64 304 294 891,98 euros et celui de la CSG imposable à -22 695 633 461,51 euros. Après application du Projet de Loi de Finances proposé par le gouvernement, ces montants restent inchangés.

Cet amendement propose de réduire le taux de la CSG déductible sur les revenus d'activité à 6,46% et le taux de la CSG imposable à 2,16%. Après cette réforme, le montant de la CSG déductible sur les salaires serait de -61 089 080 024,71 euros et celui de la CSG imposable de -20 426 070 169,84 euros.

Proposition de rédaction :

Après l'article 10, insérer l'article suivant :

"Le taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité est fixé à 6,46%. Le taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité est fixé à 2,16%."

Ces modifications seront effectives à compter du 1er janvier de l'année suivant l'adoption de la présente loi.

Les pertes de recettes pour l'État sont compensées à due concurrence par la création d'une nouvelle tranche d'imposition sur le revenu pour les revenus supérieurs à 150 000 euros.